<a href="https://colab.research.google.com/github/hwarang97/spam_classifier/blob/main/image_spam_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import cv2
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torchvision import datasets
from torchvision.datasets import ImageFolder

!pip install torchinfo
from torchinfo import summary

!pip install albumentations
import albumentations as A
from albumentations.pytorch import ToTensorV2

In [ ]:
data_dir = ''
batch_size = 32

In [5]:
transform = A.Compose([
    A.Resize(76, 76),
    A.HorizontalFlip(p=0.5),
    ToTensorV2(),
])

class CustomDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image, label = self.dataset[idx]
        image = image.numpy().transpose((1, 2, 0)) # CHW -> HWC
        if self.transform:
            image = self.transform(image=image)['image']
        return image, label

In [ ]:
dataset = datasets.ImageFolder(root=data_dir)
custom_dataset = CustomDataset(dataset, transform)
dataloader = DataLoader(dataset=custom_dataset, batch_size=batch_size, shuffle=True)

# for images, labels in dataloader:
#     ...

In [ ]:
class CNN_model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128*19*19, 128)
        self.fc2 = nn.Linear(128, 1)
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv3(x))
        x = self.dropout(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.sigmoid(self.fc2(x))
        return x

In [ ]:
model = CNN_model()
summary(model, input_size=(10, 3, 76, 76), device='cpu')

Layer (type:depth-idx)                   Output Shape              Param #
CNN_model                                [10, 1]                   --
├─Conv2d: 1-1                            [10, 32, 76, 76]          896
├─Conv2d: 1-2                            [10, 64, 38, 38]          18,496
├─Conv2d: 1-3                            [10, 128, 19, 19]         73,856
├─Dropout: 1-4                           [10, 128, 19, 19]         --
├─Linear: 1-5                            [10, 128]                 5,914,752
├─Dropout: 1-6                           [10, 128]                 --
├─Linear: 1-7                            [10, 1]                   129
Total params: 6,008,129
Trainable params: 6,008,129
Non-trainable params: 0
Total mult-adds (M): 644.60
Input size (MB): 0.69
Forward/backward pass size (MB): 25.89
Params size (MB): 24.03
Estimated Total Size (MB): 50.61